# 3 datasets库基本使用

## 基本使用
上一节中，已经介绍了使用datasets读取本地文件的方法，这一节继续介绍datasets一些常用的方法

首先是数据分割，因为我们从数据源获得DNA序列等数据，都是一个文本文件，但训练的时候，一般都需要分成训练集和测试集等

一个简单的例子如下所示：

In [1]:
#读取dna数据
from datasets import load_dataset
dna_dataset = load_dataset('text', data_files='data/dna_1g.txt')['train'].rain_test_split(test_size=0.05) #默认已经shuffle
dna_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1025615
    })
    test: Dataset({
        features: ['text'],
        num_rows: 53980
    })
})

In [2]:
dna_dataset["test"][0]

{'text': 'ATGTGTGCAATGGGTTATCTTTATGTAATAACAGTCATATCACGGGTGTTCCTCAGAAGTAGTGAACTGGCTAGCATTTTTAGACACTATGTGATCTCTCATATGACTACACTCAATTTAAAATAAAATGAAATGTGTTGTGTGTGTCTAAAATCTATAAAGGGAAAAGTATCTTAAGTATTTTTTAGATGTTAAAGTAGATGTGTATCCTAAAATATGCATTGTTCACAGATGTTAAAATTACAACTACAATCTGTGAAACACAGATCTTAGGACAGCAATGTTTCACAAGAAAAAAAATGATGCAGCCTTCTTTAGTATTTATAGTCATTTGAACAATTATGGCAACCATAAGTTCATATATAACATCCCCATTTGGTGAAACTAGTTGGGAAAGATTAGAAGGTATGACCTTGTTGGAGGAACTATACCATTGGGGTGGCTTTGAGACTTCAGAAGTTTCAAGGCCCATTTAGTGCTTTCTACCTTATGAAGCTGTGAGTTCTCCTTGCTAGCTACATAACTTGGAAAGCAGGCCCTGCACTTCACCCAAGGAGCACATTAGAGCTGGCCCTTTTGGAAGGCAATTGCGTAAGCCACACCAGGGCACCAGAGATCTGGCACTGCCATGCTCCTGCTTGCAAGTAGTGGTGTGGGTGTTGGGTGATGCCCTCCAGTCCCACCTTTTGCCACCTGTAGTAGTCAGGGGAGTTGGCCTAAGGGCATGAGAGCCTAAGACTTCACCCTAATCCCTCACCAACTGTAGCATGTGGAAGAGCAGGCTCTGTACCTTCCCTGGGCAACACATTGGAGCTGGCCCCTCACAGGCTGCAGGACTTGGGAGAGTGAGTGCTGCACCTTGACTGTGAAGGTGGTTTTGGAGGTGTGGGTGTGAGACCATGAGACCAAGAGAGGAATGGAATATTACTCACTTATTAAAAACAATGACTTCATGAAATTTGCAGGCAAATGGATGGAACTTGAAAATAT

可以看到，数据集已经分割成了train和test两个数据集，而在分割的时候，已经进行的随机处理

当然，如果数据集过大，我们只需要其中一部分，这个也是一个常见的需求，一般可以使用 Dataset.select() 函数

In [4]:
from datasets import load_dataset, DatasetDict

dna_dataset_sample =  DatasetDict(
    {
        "train": dna_dataset["train"].shuffle().select(range(50000)), 
        "valid": dna_dataset["test"].shuffle().select(range(500)),
        "evla": dna_dataset["test"].shuffle().select(range(500))

    }
)
dna_dataset_sample

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 50000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 500
    })
})

可以看到，我们使用DatasetDict来直接构造datasets，先使用shuffle()来随机，然后使用select来选择前n个数据

select的参数为indices (list 或 range): 索引列表或范围对象，指明要选择哪些样本，如dataset.select([0, 2, 4])就是选择1,3,5条记录

## 读取线上库

当然，数据也可以直接从huggingface的线上仓库读取，这时候需要注意科学上网问题。

具体使用函数也是load_dataset

<img src='img/datasets_dnagpt.png' width='800px' />

In [9]:
import subprocess
import os
# 设置环境变量, autodl一般区域
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

#或者
"""
import os

# 设置环境变量, autodl专区 其他idc
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))
"""

"\nimport os\n\n# 设置环境变量, autodl专区 其他idc\nos.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'\n\n# 打印环境变量以确认设置成功\nprint(os.environ.get('HF_ENDPOINT'))\n"

In [10]:
from datasets import load_dataset
dna_data = load_dataset("dnagpt/dna_core_promoter")
dna_data

DatasetDict({
    train: Dataset({
        features: ['sequence', 'label'],
        num_rows: 59196
    })
})

当然，如果你想分享你的数据集到huggingface上面，也是一行函数即可：

In [ ]:
dna_data.push_to_hub("org_name/your_dataset_name", token="hf_yourtoken")